### NOTE! HASHTAGGED CELLS THAT HAVE FORMULAS SHOULD  HAVE THE # REMOVED BEFORE RUNNING *security reasons*

In [1]:
# -*- coding: utf-8 -*-


"""
Created on Tue Apr  6 17:35:01 2021

@author: schne
"""
import re
import pandas as pd
from bs4 import BeautifulSoup
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import email
import base64
import requests

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def search_message(service, user_id, search_string):
    """
    Search the inbox for emails using standard gmail search parameters
    and return a list of email IDs for each result
    PARAMS:
        service: the google api service object already instantiated
        user_id: user id for google api service ('me' works here if
        already authenticated)
        search_string: search operators you can use with Gmail
        (see https://support.google.com/mail/answer/7190?hl=en for a list)
    RETURNS:
        List containing email IDs of search query
    """
    try:
        # initiate the list for returning
        list_ids = []

        # get the id of all messages that are in the search string
        search_ids = service.users().messages().list(userId=user_id, q=search_string).execute()
        
        # if there were no results, print warning and return empty string
        try:
            ids = search_ids['messages']

        except KeyError:
            print("WARNING: the search queried returned 0 results")
            print("returning an empty string")
            return ""

        if len(ids)>1:
            for msg_id in ids:
                list_ids.append(msg_id['id'])
            return(list_ids)

        else:
            list_ids.append(ids['id'])
            return list_ids
        
    except (errors.HttpError, error):
        print("An error occured: %s") % error
        
        
def get_message(service, user_id, msg_id):
    """
    Search the inbox for specific message by ID and return it back as a 
    clean string. String may contain Python escape characters for newline
    and return line. 
    
    PARAMS
        service: the google api service object already instantiated
        user_id: user id for google api service ('me' works here if
        already authenticated)
        msg_id: the unique id of the email you need
    RETURNS
        A string of encoded text containing the message body
    """
    try:
        # grab the message instance
        message = service.users().messages().get(userId=user_id, id=msg_id,format='raw').execute()

        # decode the raw string, ASCII works pretty well here
        msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))

        # grab the string from the byte object
        mime_msg = email.message_from_bytes(msg_str)

        # check if the content is multipart (it usually is)
        content_type = mime_msg.get_content_maintype()
        if content_type == 'multipart':
            # there will usually be 2 parts the first will be the body in text
            # the second will be the text in html
            parts = mime_msg.get_payload()

            # return the encoded text
            final_content = parts[0].get_payload()
            return final_content

        elif content_type == 'text':
            return mime_msg.get_payload()

        else:
            return ""
            print("\nMessage is not text or multipart, returned an empty string")
    # unsure why the usual exception doesn't work in this case, but 
    # having a standard Exception seems to do the trick
    except Exception:
        print("An error occured: %s") % error


def get_service():
    """
    Authenticate the google api client and return the service object 
    to make further calls
    PARAMS
        None
    RETURNS
        service api object from gmail for making calls
    """
    
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    return service

### Method to Scrape Risk Range Table from the Hedgeye website (currently unable to use as no HTTP access :(

r = requests.get("https://app.hedgeye.com/users/sign_in", auth=('', ''))

r

r.headers['content-type']

r.encoding

r.text

s = requests.get('')

s.text

soup3=BeautifulSoup(r,'html.parser')

In [2]:
service=get_service()
#search_message(service,'me','Risk Ranges')

['178b6a1da33dc736',
 '178b675562417123',
 '178b166d5d3bed46',
 '178ac35ef5b46790',
 '178a7278799e5f84',
 '178a7188cb49c5ae',
 '178a1f2520c7d299',
 '17898bc37bf8bce9',
 '178987cf8a4139a9',
 '17897e0d8c5d26d8',
 '1788d4e06e8efd8f',
 '178882dc6ba33b38',
 '178851e095a5d20a',
 '1788333745b0f4f6',
 '17882fe62d70cb4a',
 '1787dcf1ca93c3b2',
 '178799d3e1b62928',
 '1786e624bc818228',
 '1786946ed378be54',
 '1786417c71c9df57',
 '1785ef15e49e0838',
 '1785b986e1c0d996',
 '1785b930d36c5b8c',
 '1785b9018af71aa6',
 '1785b8da58ffd760',
 '1785a7e905a5beec',
 '17859e5cd100528a',
 '17859c8091567e0f',
 '1785525b2d76e453',
 '1784a47b8ac0fab4',
 '17845ec1d1827942',
 '1784564438077c12',
 '1784555a057cf845',
 '178453d33b1972e9',
 '17840163c9aaa5dc',
 '1783af6421a6d9cb',
 '17835bf5fd97d5b3',
 '17830e4acbd81440',
 '17826a85ca0f473a',
 '178219f870d27c90',
 '178219f09b7d7002',
 '178215f1ee928c6e',
 '1781c54edd98a1cf',
 '1781c4c4f0514551',
 '1781c3a242a90c02',
 '1781c38942ac6a63',
 '1781731277e2bdde',
 '178171616fb

In [3]:
test=get_message(service,'me','178ac35ef5b46790')


## Beautiful Soup
soup = BeautifulSoup(test, 'lxml')

#print(soup.prettify())

rr=soup.p.contents[1]
rr=rr.contents[1]
#rr.contents

#### Grab Dates From Email Message

In [4]:
#Convert soup to unicode
unicode_string=str(rr)
#print(unicode_string)

#Find date indicies in email
#print(unicode_string.find("04")) ##Enter the date for intial setup, then this line can be removed

#grab date string
rr_dates=unicode_string[42:50]
rr_dates

'04/07/21'

#### Grab RRs From Email Message

In [5]:
#Find RR indicies in email
#print(unicode_string.find("Bullish"))
#print(unicode_string.find("Hedgeye's Risk Ranges"))


#grab rr string
rr_rrs=unicode_string[231:2063]
#print(type(rr_rrs))
rr_rrs

'Bullish Trend Bearish Trend Neutral Trend\r\n\r\nINDEX BUY TRADE SELL TRADE PREV. CLOSE UST10Y (BULLISH)\r\n10-Year U.S. Treasury Yield 1.79 1.60 1.67 UST2Y (BULLISH)\r\n2-Year U.S. Treasury Yield 0.18 0.13 0.16 SPX (BULLISH)\r\nS&amp;P 500 3,980 4,115 4,073 RUT (BULLISH)\r\nRussell 2000 2,125 2,299 2,259 COMPQ (BULLISH)\r\nNASDAQ Composite 13,058 13,828 13,698 XLK (BULLISH)\r\nTechnology Select Sector SPDR Fund 131.56 139.51 137.69 XLE (BULLISH)\r\nEnergy Select Sector SPDR Fund 48.02 51.55 48.98 XLF (BULLISH)\r\nFinancials Select Sector SPDR Fund 33.50 35.15 34.67 XLU (BEARISH)\r\nUtilities Select Sector SPDR Fund 62.27 65.30 65.02 SSEC (BEARISH)\r\nShanghai Composite 3,340 3,510 3,479 NIKK (BULLISH)\r\nNikkei 225 Index 28,755 30,202 29,730 DAX (BULLISH)\r\nGerman DAX Composite 14,816 15,298 15,212 VIX (BEARISH)\r\nVolatility Index 16.58 20.93 18.12 USD (BEARISH)\r\nU.S. Dollar Index 91.66 93.39 92.35 EUR/USD (BULLISH)\r\nEuro to U.S. Dollar 1.169 1.196 1.188 USD/JPY (BULLISH)\r\nU.

In [6]:
test_split = rr_rrs.split('\r\n')
test_split

['Bullish Trend Bearish Trend Neutral Trend',
 '',
 'INDEX BUY TRADE SELL TRADE PREV. CLOSE UST10Y (BULLISH)',
 '10-Year U.S. Treasury Yield 1.79 1.60 1.67 UST2Y (BULLISH)',
 '2-Year U.S. Treasury Yield 0.18 0.13 0.16 SPX (BULLISH)',
 'S&amp;P 500 3,980 4,115 4,073 RUT (BULLISH)',
 'Russell 2000 2,125 2,299 2,259 COMPQ (BULLISH)',
 'NASDAQ Composite 13,058 13,828 13,698 XLK (BULLISH)',
 'Technology Select Sector SPDR Fund 131.56 139.51 137.69 XLE (BULLISH)',
 'Energy Select Sector SPDR Fund 48.02 51.55 48.98 XLF (BULLISH)',
 'Financials Select Sector SPDR Fund 33.50 35.15 34.67 XLU (BEARISH)',
 'Utilities Select Sector SPDR Fund 62.27 65.30 65.02 SSEC (BEARISH)',
 'Shanghai Composite 3,340 3,510 3,479 NIKK (BULLISH)',
 'Nikkei 225 Index 28,755 30,202 29,730 DAX (BULLISH)',
 'German DAX Composite 14,816 15,298 15,212 VIX (BEARISH)',
 'Volatility Index 16.58 20.93 18.12 USD (BEARISH)',
 'U.S. Dollar Index 91.66 93.39 92.35 EUR/USD (BULLISH)',
 'Euro to U.S. Dollar 1.169 1.196 1.188 USD/J

In [7]:
test_series=pd.Series(test_split)
test_series.head()

0            Bullish Trend Bearish Trend Neutral Trend
1                                                     
2    INDEX BUY TRADE SELL TRADE PREV. CLOSE UST10Y ...
3    10-Year U.S. Treasury Yield 1.79 1.60 1.67 UST...
4    2-Year U.S. Treasury Yield 0.18 0.13 0.16 SPX ...
dtype: object

In [8]:
##Break out DataFrame Columns from Series Data
cols=test_series[2:3]
#print(cols)

header = [n.split(' ') for n in cols]
#print(header)

header_df=pd.DataFrame(header)
#print(header_df)

header_df.columns = header_df.iloc[0]

header_df=header_df.drop(header_df.index[0])

#print(header_df)

##Grabbing the mislocated Trend value for implementation back into dataframe later on
trend_header=header_df.columns[-1]
#print(trend_header)
##Grabbing the mislocated Security Name (UST10 YR) for implementation back into dataframe later on
sec_header=header_df.columns[-2]
#print(sec_header)

#### So we have some excess columns that we need to get rid of, and we also need to save this information into it's own dataframe b/c it is actually data, and not data attributes or descriptions

In [9]:
### Drop excess columns (We'll figure our later how to save these values into their own dataframe for easy appending to final dataframe)

#### THIS CODE HAS APPENDEGAGES!!!!, if need to access old header_df, rerurn the code ABOVE THIS CELL
header_cols=[-1,-2]
header_df.drop(header_df.columns[header_cols],axis=1,inplace=True)
header_df

,INDEX,BUY,TRADE,SELL,TRADE,PREV.,CLOSE


In [10]:
raw=test_series.reset_index(drop=True)
raw=raw[3:].reset_index(drop=True)
raw.head()

0    10-Year U.S. Treasury Yield 1.79 1.60 1.67 UST...
1    2-Year U.S. Treasury Yield 0.18 0.13 0.16 SPX ...
2          S&amp;P 500 3,980 4,115 4,073 RUT (BULLISH)
3       Russell 2000 2,125 2,299 2,259 COMPQ (BULLISH)
4    NASDAQ Composite 13,058 13,828 13,698 XLK (BUL...
dtype: object

In [11]:
## Replace (remove) spaces between indexes that do not work with regex logic (ie Russell 2000, Nikkei 225, SP 500, etc)
raw=raw.str.replace(r"(?<=[liP]) (?=\d)",'') ## searching for first instances of '1 str' + ' ' + '1 int'
raw=raw.str.replace(r"(?<=\d) (?=[I])",'') ## Regex specific to Nikkei 225 Index
raw.head()

<ipython-input-11-cdaf718a571c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  raw=raw.str.replace(r"(?<=[liP]) (?=\d)",'') ## searching for first instances of '1 str' + ' ' + '1 int'
<ipython-input-11-cdaf718a571c>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  raw=raw.str.replace(r"(?<=\d) (?=[I])",'') ## Regex specific to Nikkei 225 Index


0    10-Year U.S. Treasury Yield 1.79 1.60 1.67 UST...
1    2-Year U.S. Treasury Yield 0.18 0.13 0.16 SPX ...
2           S&amp;P500 3,980 4,115 4,073 RUT (BULLISH)
3        Russell2000 2,125 2,299 2,259 COMPQ (BULLISH)
4    NASDAQ Composite 13,058 13,828 13,698 XLK (BUL...
dtype: object

In [12]:
##Begin Splitting series for compatible DataFrame creation
raw = raw.str.split(r"(?![a-z]) (?=\d)", expand=True)
raw.head()

,0,1,2,3
0,10-Year U.S. Treasury Yield,1.79,1.60,1.67 UST2Y (BULLISH)
1,2-Year U.S. Treasury Yield,0.18,0.13,0.16 SPX (BULLISH)
2,S&amp;P500,"3,980","4,115","4,073 RUT (BULLISH)"
3,Russell2000,"2,125","2,299","2,259 COMPQ (BULLISH)"
4,NASDAQ Composite,"13,058","13,828","13,698 XLK (BULLISH)"


In [13]:
squoze_raw = raw.iloc[:,-1:].squeeze()
#print(squoze_raw.head())

lrs=squoze_raw.str.split(r"(?<=\d) (?=[A-Z])", expand=True)
lrs.tail()

,0,1
29,544,TSLA (BEARISH)
30,691,NYXBT (BULLISH)
31,"57,744",None
32,None,None
33,None,None


In [14]:
squoze_lrs= lrs.iloc[:,-1:].squeeze()
#print(len(squoze_lrs))
squoze_lrs.tail()

29     TSLA (BEARISH)
30    NYXBT (BULLISH)
31               None
32               None
33               None
Name: 1, dtype: object

In [15]:
final_split=squoze_lrs.str.split(r"(?<=[A-Z]) (?=[(])" , expand=True)
ticker_split = final_split.iloc[:,0]
ticker_split=ticker_split.reset_index(drop=True)
#print(len(ticker_split))
#print(ticker_split.tail())


trend_split=final_split.iloc[:,-1:]
trend_split=trend_split.reset_index(drop=True)
#print(len(trend_split))
#print(trend_split.tail())

In [16]:
### Adding the Trend Value that was originally located in the df header

#Shift whole column down 
trend_split=trend_split.shift(periods=1,axis=0)

#Add trend value to new empty cell
trend_split.iloc[0]=trend_header
trend_split.tail()

,1
29,(BULLISH)
30,(BEARISH)
31,(BULLISH)
32,None
33,None


In [17]:
### Adding the Ticker Value that was originally located in the df header

#Shift whole column down 
ticker_split=ticker_split.shift(periods=1,axis=0)

#Add trend value to new empty cell
ticker_split.iloc[0]=sec_header
ticker_split.tail()

29     NFLX
30     TSLA
31    NYXBT
32     None
33     None
Name: 0, dtype: object

In [18]:
final_df = raw.assign(LastPrice = lrs[0])

final_df = final_df.assign(Trend = trend_split)
final_df = final_df.assign(Ticker = ticker_split)

final_df = final_df.drop([3], axis =1)
final_df.rename(columns={0: "Asset", 1: "LERR",2: "TERR"})
final_df.head()

,0,1,2,LastPrice,Trend,Ticker
0,10-Year U.S. Treasury Yield,1.79,1.60,1.67,(BULLISH),UST10Y
1,2-Year U.S. Treasury Yield,0.18,0.13,0.16,(BULLISH),UST2Y
2,S&amp;P500,"3,980","4,115","4,073",(BULLISH),SPX
3,Russell2000,"2,125","2,299","2,259",(BULLISH),RUT
4,NASDAQ Composite,"13,058","13,828","13,698",(BULLISH),COMPQ


In [19]:
final_df['Date']=rr_dates
final_df

,0,1,2,LastPrice,Trend,Ticker,Date
0,10-Year U.S. Treasury Yield,1.79,1.60,1.67,(BULLISH),UST10Y,04/07/21
1,2-Year U.S. Treasury Yield,0.18,0.13,0.16,(BULLISH),UST2Y,04/07/21
2,S&amp;P500,"3,980","4,115","4,073",(BULLISH),SPX,04/07/21
3,Russell2000,"2,125","2,299","2,259",(BULLISH),RUT,04/07/21
4,NASDAQ Composite,"13,058","13,828","13,698",(BULLISH),COMPQ,04/07/21
5,Technology Select Sector SPDR Fund,131.56,139.51,137.69,(BULLISH),XLK,04/07/21
6,Energy Select Sector SPDR Fund,48.02,51.55,48.98,(BULLISH),XLE,04/07/21
7,Financials Select Sector SPDR Fund,33.50,35.15,34.67,(BULLISH),XLF,04/07/21
8,Utilities Select Sector SPDR Fund,62.27,65.30,65.02,(BEARISH),XLU,04/07/21
9,Shanghai Composite,"3,340","3,510","3,479",(BEARISH),SSEC,04/07/21


From here, these ranges can be grabbed daily and uploaded to a csv or excel file. The process of loading is not included in this repository because the same logic is in another one of my repositories and I wanted to get this out quickly for the #HedgeyeNation. 